# INF367 Assignment 1, Exercise 2
Author: Johanna Jøsang (fak006)

In [84]:
import numpy as np

In [89]:
# Arguments
n = 10 #number of variables
delta = 0.1
eps = 0.1

ten_div_delta = int(10/delta)
m = int(((2*n) / eps) * ((np.log(2*n) + np.log(1/delta))))
N = int((20/eps)**2)


## Notes about methods used for PAC implementation
Since a hypothesis only can be given positive examples if the target is satisfiable, only satisfiable targets are used in this implementation. A satisfiable target is generated by combining data generated using a normal distribution. "1" indicates that a variable is evaluated to True, "0" that is is evaluated to False. The target does not necessarily contain all literals, hence it may contain "3" which indicated that this variable is not present in the target. \
Target generation example:\
[0, 1, 1, 0, 0, 0, 1, 1, 0, 0] <- basis 1 and \
[0, 1, 1, 0, 1, 0, 0, 0, 0, 1] <- basis 2 generate:\
[0, 1, 1, 0, 3, 0, 3, 3, 0, 3] <- target

Examples are generated in the same way as bases for the target, by normal distribution. Each example is categorized by whether it satisfies the target or not. The following examples are based on the target given above. Notice that the valuation of a variable not present in the target has no influence on the categorization.

([0, 1, 1, 1, 0, 0, 1, 1, 0, 1],  0) <- valuation that doesn't satisfy target \
([0, 1, 1, 0, 1, 0, 1, 0, 0, 0],  1) <- valuation that does satisfy target 

Before training, the hypothesis looks like this [2, 2, 2, 2, 2, 2, 2, 2, 2, 2], where the "2" indicates that both the variable and its negation are evaluated to true.
Here is a depiction of a training iteration, where only two positive examples were given.
H: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2] <- initial hypothesis \
  ([0, 1, 1, 0, 1, 0, 1, 0, 0, 0], 1) <- positive example 1 \
H: [0, 1, 1, 0, 1, 0, 1, 0, 0, 0]  <- hypothesis after updating from example 1 \
  ([0, 1, 1, 0, 0, 0, 1, 0, 0, 0], 1) <- positive example 2 \
H: [0, 1, 1, 0, 3, 0, 1, 0, 0, 0] <- hypothesis after updating from example 2 

If we compare the target and the final hypothesis we can see the result. \
Target-----: [0, 1, 1, 0, 3, 0, 3, 3, 0, 3] \
Trained hyp: [0, 1, 1, 0, 3, 0, 1, 0, 0, 0] 

The hypothesis has not quite reached the target.

In [90]:

def generate_uncategorized_data(size, dimensions):
    """
    Using a random normal distribution, 2D datasets of 1s and 0s are generated
    
    :param size: number of arrays to generate
    :param dimensions: length of each array
    :return : 2D array of uncategorized data
    
    """
    normal_distribution = np.random.normal(size=[size, dimensions]) >= 0
    normal_distribution_ints = normal_distribution.astype(int)                        
    return normal_distribution_ints.tolist()

def generate_satisfiable_target(size, dimensions):
    """
    Using a random normal distribution, 2D datasets of 1s and 0s are generated
    
    :param size: number of arrays to generate
    :param dimensions: length of each array
    
    """
    target_basis_ints = generate_uncategorized_data(size, dimensions)
    target = [1]*(len(target_basis_ints[0]))
    for j in range (len(target_basis_ints[0])):
        column_target_value = 3
        column_bool = target_basis_ints[0][j]
        use_bool = True
        for i in range (len(target_basis_ints)):
            if (target_basis_ints[i][j] != column_bool):
                column_target_value = 3
                use_bool = False
                continue
        if(use_bool):
            if(column_bool == True):
                column_target_value = 1
            else:
                column_target_value = 0
        target[j] = column_target_value#
    return target
    
def categorize_data(valuation, target):
    """
    Categorizes valuations based on if they satisfy the target   
    
    :param valuation: valuation to categorize
    :param target: used classify the valuation
    :return : 1 if the valuation satisfies the target, 0 otherwise
    
    """
    if(len(valuation) != len(target)):
        print('data length:', str(len(valuation)), ' != target length', str(len(target)))
        return 4 # this should never happen, hence and invalid number is given
    for i in range (len(target)):
        #if t[i] = 3, d[i] = 2 or t[i] == d[i], d entails t
        if((target[i] != 3) and (valuation[i] != 2) and (target[i] != valuation[i])):
                return 0 # valuation does not satisfy target
    return 1 # valuation does satisfy target

def generate_examples(data_list, target):
    """
    Generates a list of tupples where each set of valuations is paired with a classification, ie a positive or negative example.
    
    :param data: set of uncategorized examples
    :param data: 
    :return : list of categorized valuations
    """
    examples = []
    for valuation in data_list:
        category = categorize_data(valuation, target)
        examples.append((valuation, category))
    return examples

def initialize_hypothesis(length):
    """
    Generates a initial hypothesis of a certian length
    
    :param length: length of hypothesis to generate
    :return : initial hypothesis in the form of a list of 2s
    """
    return [2] * length

def train_hypothesis(training_set, hypothesis):
    """
    Uses the algorithm for learning a conjunction of literals. Positive examples in the training set are used to update the hypothesis.
    
    Uncomment the commented lines to read information about the training.
    
    :param training_set: list of categorized valuations
    :param hypothesis: valuation to update
    :return : trained hypothesis
    """
    #n_of_pos = 0
    for e in training_set:
        if (e[1] == 1):
            # print(e)
            #n_of_pos = n_of_pos + 1
            e_variables = e[0]
            for i in range(len(e_variables)):
                if ((e_variables[i] != hypothesis[i]) and (hypothesis[i] != 3)):
                    if(hypothesis[i] == 2):
                        hypothesis[i] = e_variables[i]
                    else:
                        hypothesis[i] = 3
            #print('H:', hypothesis)
    #print('Pos exampes / set size: ', n_of_pos, '/', len(training_set))                    
    #print('Target:            ', target)
    #print('Trained hypothesis:', hypothesis)
    return hypothesis

def disjoint_union(mt, mh):
    """
    Generated the disjoint union between two sets.
    
    :param mt: set of valuations that satisfy the target
    :param mh: det of valuations that satisfy the hypothesis
    :return : disjoint union between mt and mh
    """
    for e in mt:
        if e in mh:
            mt.remove(e)
            mh.remove(e)
    for e in mh:
        if e in mt:
            mt.remove(e)
            mh.remove(e)
    return mt + mh

def calculate_t_hat(S, target, hypothesis):
    """
    Calculates the estimated error of the trained hypothesis w.r.t the target. 
    
    :param S: set of unlabeled examples
    :param target: to estimate the error w.r.t
    :param hypothesis: to estimate the error of
    """
    mt = [] #set of valuations in S that satisfy the target
    mh = [] #set of valuations in S that satisfy the target
    for s in S:
        if (categorize_data(s, target) == 1):
            mt.append(s)
        if (categorize_data(s, hypothesis) == 1):
            mh.append(s)
    error_s = disjoint_union(mt, mh) #find misclassified valuations
    t_hat = len(error_s) / len(S) # error = (no. misclassified valuations)/(total no. of valuations)
    return t_hat

## PAC implementation

In [91]:
# Print out arguments being used
print('Number of variables used:', n)
print('Epsilon:', eps)
print('Delta:', delta)
print('m:', m)
print('10/delta:', ten_div_delta)
print('N:', N)
print()

print("Number of iterations:", ten_div_delta)
bad_error_count = 0 # number of errors larger than epsolin
for i in range(ten_div_delta):    
    target = generate_satisfiable_target(2,n) # we generate a target with 2 bases
    uncategorized_data = generate_uncategorized_data(m, n)
    examples = generate_examples(uncategorized_data, target) #generate examples
    h = initialize_hypothesis(n)
    trained_h = train_hypothesis(examples, h) #train the hypothesis
    unlabeled_ex =  generate_uncategorized_data(N, n)
    t = calculate_t_hat(unlabeled_ex, target, trained_h) #calculate the error
    if (t > eps):
        bad_error_count = bad_error_count + 1 # count the number of errors greater than epsilon
print('Number of errors greater than epsilon:', bad_error_count)
print('Bad error rate:', bad_error_count/ten_div_delta)


Number of variables used: 10
Epsilon: 0.1
Delta: 0.1
m: 1059
10/delta: 100
N: 40000

Number of iterations: 100
Number of errors greater than epsilon: 5
Bad error rate: 0.05
